In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
path = 'D:\data\catdog\data\\'
train_file = {0: [], 1: []}
test_file = {0: [], 1: []}

for _, _, files in os.walk(path):
    for file in files:
        if file.startswith('test'):
            if file.endswith('0.csv'):
                test_file[0].append(file)
            else : test_file[1].append(file)
        else :
            if file.endswith('0.csv'):
                train_file[0].append(file)
            else : train_file[1].append(file)
print(train_file, test_file)

train_data = {0: None, 1: None}
test_data = {0: None, 1: None}

for i in ['train', 'test']:
    for key, val in eval(i + '_file').items():
        for file in val :
            print(i, key, file)
            temp = np.loadtxt(path + file, delimiter=',') / 255
            temp = np.hstack((temp, np.ones((temp.shape[0], 1))*key))
            temp_name = eval(i + '_data')
            if temp_name[key] is None :
                temp_name[key] = temp
            else :
                temp_name[key] = np.vstack((temp_name[key], temp))
            print(temp_name[key].shape)

{0: ['train_data_1_0.csv', 'train_data_2_0.csv', 'train_data_3_0.csv', 'train_data_4_0.csv', 'train_data_5_0.csv'], 1: ['train_data_11_1.csv', 'train_data_12_1.csv', 'train_data_13_1.csv', 'train_data_14_1.csv', 'train_data_15_1.csv']} {0: ['test_data_1_0.csv', 'test_data_2_0.csv', 'test_data_3_0.csv'], 1: ['test_data_7_1.csv', 'test_data_8_1.csv', 'test_data_9_1.csv']}
train 0 train_data_1_0.csv
(1000, 16385)
train 0 train_data_2_0.csv
(2000, 16385)
train 0 train_data_3_0.csv
(3000, 16385)
train 0 train_data_4_0.csv
(4000, 16385)
train 0 train_data_5_0.csv
(5000, 16385)
train 1 train_data_11_1.csv
(862, 16385)
train 1 train_data_12_1.csv
(1862, 16385)
train 1 train_data_13_1.csv
(2862, 16385)
train 1 train_data_14_1.csv
(3862, 16385)
train 1 train_data_15_1.csv
(4862, 16385)
test 0 test_data_1_0.csv
(1000, 16385)
test 0 test_data_2_0.csv
(2000, 16385)
test 0 test_data_3_0.csv
(3000, 16385)
test 1 test_data_7_1.csv
(1000, 16385)
test 1 test_data_8_1.csv
(2000, 16385)
test 1 test_data_9

In [31]:
x = tf.placeholder('float', shape=[None, 128 * 128], name='images_flat')
y_ = tf.placeholder(tf.int64, shape=[None], name='labels')

In [32]:
x_image = tf.reshape(x, [-1, 128, 128, 1], name='batch_images')
print(x_image)

Tensor("batch_images_1:0", shape=(?, 128, 128, 1), dtype=float32)


In [33]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(.1, shape=shape)
    return tf.Variable(initial)

In [34]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [35]:
W_conv1 = weight_variable([5, 5, 1, 16])
b_conv1 = bias_variable([16])

In [36]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [37]:
W_conv2 = weight_variable([5, 5, 16, 32])
b_conv2 = bias_variable([32])

In [38]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [39]:
W_fc1 = weight_variable([32 * 32 * 32, 1024])
b_fc1 = bias_variable([1024])

In [40]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 32 * 32 * 32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [41]:
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [42]:
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

In [49]:
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [50]:
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [51]:
sess = tf.Session()

In [52]:
sess.run(tf.global_variables_initializer())

In [53]:
batch_0, batch_1 = np.random.choice(train_data[0].shape[0], 50), np.random.choice(train_data[1].shape[0], 50)
batch = np.vstack((train_data[0][batch_0], train_data[1][batch_1]))
np.random.shuffle(batch)
sess.run(tf.to_int32(batch[0, -1]))

0

In [54]:
for i in range(1000):
    batch_0, batch_1 = np.random.choice(train_data[0].shape[0], 50), np.random.choice(train_data[1].shape[0], 50)
    batch = np.vstack((train_data[0][batch_0], train_data[1][batch_1]))
    np.random.shuffle(batch)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: batch[:, :-1], y_: batch[:, -1], keep_prob: 1.})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    sess.run(train_step, feed_dict={x: batch[:, :-1], y_: batch[:, -1], keep_prob: .5})

step 0, training accuracy 0.46
step 100, training accuracy 0.72
step 200, training accuracy 0.76
step 300, training accuracy 0.8
step 400, training accuracy 0.73
step 500, training accuracy 0.78
step 600, training accuracy 0.82
step 700, training accuracy 0.79
step 800, training accuracy 0.84
step 900, training accuracy 0.94


In [60]:
test = np.vstack((test_data[0], test_data[1]))
np.random.shuffle(test)
print('test accuracy %g' % 
      sess.run(accuracy, feed_dict={x: test[:100, :-1], y_: test[:100, -1], keep_prob: 1.}))

test accuracy 0.65
